In [95]:
import random
import csv
import pandas as pd
import tenseal as ts
import numpy as np

In [105]:
def generate_minutiae(num_fingerprints, num_minutiae_per_fingerprint):
    dataset = []
    for i in range(num_fingerprints):
        fingerprint = []
        fingerprint_id = i+1
        for _ in range(num_minutiae_per_fingerprint):
            x = random.randint(0, 500) # x coord
            y = random.randint(0, 500) # y coord
            angle = random.randint(0, 360) # angle of minutiae
            minutiae_type = random.randint(0, 1) #0 for ridge, 1 for bif
            fingerprint.append((fingerprint_id, x, y, angle, minutiae_type)) # add to individual fingerprint data
        dataset.append(fingerprint)
    return dataset

def save_to_csv(dataset, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ID', 'X', 'Y', 'Angle', 'Type'])
        for fingerprint in dataset:
            for minutiae in fingerprint:
                writer.writerow(minutiae)

def encrypt_fingerprint_data(df, context):
    encrypted_minutiae_dict = {}
    for fingerprint_id in df['ID'].unique():
        fingerprint_minutiae = df[df['ID'] == fingerprint_id][['X', 'Y', 'Angle', 'Type']].values.flatten().tolist()
        scaled_minutiae = [float(value) for value in fingerprint_minutiae]
        encrypted_minutiae = ts.ckks_vector(context, scaled_minutiae)
        encrypted_minutiae_dict[fingerprint_id]=encrypted_minutiae
    return encrypted_minutiae_dict

def find_matching_fingerprint_id(encrypted_query, encrypted_data, context):
    min_distance = float('inf')
    matching_id = None

    for fingerprint_id, encrypted_minutiae in encrypted_data.items():
        difference_vector = encrypted_query - encrypted_minutiae
        distance_squared = difference_vector.dot(difference_vector)

        distance_squared_decrypted = distance_squared.decrypt()
        distance = np.sqrt(distance_squared_decrypted)

        if distance < min_distance:
            min_distance = distance
            matching_id = fingerprint_id

    return matching_id

def normalize_and_scale_minutiae(df):
    # Example normalization - adjust based on your data's characteristics
    df['X'] = df['X'] / df['X'].max()
    df['Y'] = df['Y'] / df['Y'].max()
    df['Angle'] = df['Angle'] / 360
    # No change for 'Type' as it's categorical (0 or 1)
    return df

#### Generate Random Dataset of Fingerprint minutiae

In [107]:
dataset = generate_minutiae(10, 20) # num fingerprints, minutiae per print
save_to_csv(dataset, 'fingerprint_test.csv')
df = pd.read_csv('fingerprint_test.csv')
df_normalized = normalize_and_scale_minutiae(df)
df_normalized

#### Setup TenSEAL context

In [109]:
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384,
    coeff_mod_bit_sizes=[60, 40, 40, 40, 60]
)

context.global_scale = 2**40
context.generate_galois_keys()
context.generate_relin_keys()
secret_key = context.secret_key()

#### Single print encryption for comparison testing

In [ ]:

fingerprint_id = 1
fingerprint_minutiae = df[df['ID']==fingerprint_id][['X', 'Y', 'Angle', 'Type']].values.flatten().tolist()
scaled_minutiae = [float(value) for value in fingerprint_minutiae]

fingerprint_id = 2
fingerprint_minutiae2 = df[df['ID']==fingerprint_id][['X', 'Y', 'Angle', 'Type']].values.flatten().tolist()
scaled_minutiae2 = [float(value) for value in fingerprint_minutiae2]

In [110]:
encrypted_minutiae = ts.ckks_vector(context, scaled_minutiae)
encrypted_minutiae2 = ts.ckks_vector(context, scaled_minutiae2)
encrypted_minutiae3 = ts.ckks_vector(context, scaled_minutiae)

#### encrypt entire normalized dataset

In [111]:
encrypted_data = encrypt_fingerprint_data(df_normalized, context)

In [102]:
distance_squared = (encrypted_minutiae-encrypted_minutiae2).dot(encrypted_minutiae-encrypted_minutiae2) # dot product distance between two encrypted datapoints
distance_squared_decrypted = distance_squared.decrypt() # decrypt distance to test similarity (should be large value -> fail)

distance_squared2 = (encrypted_minutiae-encrypted_minutiae3).dot(encrypted_minutiae-encrypted_minutiae3) # same as above
distance_squared_decrypted2 = distance_squared2.decrypt() # decrypt distance (should be very small value -> success)

print("Squared Euclidean distance (decrypted for demo):", distance_squared_decrypted)
print(distance_squared_decrypted2)

Squared Euclidean distance (decrypted for demo): [2147889.2880517947]
[-8.373405442525354e-07]


#### test single print data against entire encrypted data and return ID (doesn't work 100% correctly yet)

In [114]:
id_match = find_matching_fingerprint_id(encrypted_minutiae, encrypted_data, context)
print(id_match)

C:\Users\quinn\AppData\Local\Temp\ipykernel_26180\3742005205.py:41: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt(distance_squared_decrypted)


3
